# Get the model metadata from ModelMesh Server via GRPC

#### Set grpc endpoint and model name

In [ ]:
!pip install grpcio grpcio-tools 

In [ ]:
grpc_host = "modelmesh-serving"
grpc_port = 8033
model_name = "stocks"

#### Imports

In [ ]:
# grpc_predict_v2_pb2 and grpc_predict_v2_pb2_grpc were created from grpc_predict_v2.proto using protoc
import grpc
import grpc_predict_v2_pb2
import grpc_predict_v2_pb2_grpc

#### Create grpc channel and stub

In [ ]:
channel = grpc.insecure_channel(f"{grpc_host}:{grpc_port}")
stub = grpc_predict_v2_pb2_grpc.GRPCInferenceServiceStub(channel)

### Model Metadata
The per-model metadata API provides information about a model. Errors are indicated by the google.rpc.Status returned for the request. The OK code indicates success and other codes indicate failure.

In [ ]:
request = grpc_predict_v2_pb2.ModelMetadataRequest(name=model_name)
response = stub.ModelMetadata(request)
print(response)